In [1]:
import pandas as pd
import numpy as np
dc_listings = pd.read_csv('listings.csv')

In [2]:
# Display setting for viewing (to view all the columns)
pd.set_option('display.max_columns', 999) 

In [3]:
dc_listings.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,7087327,https://www.airbnb.com/rooms/7087327,20151002231825,2015-10-03,Historic DC Condo-Walk to Capitol!,Professional pictures coming soon! Welcome to ...,NaN,Professional pictures coming soon! Welcome to ...,none,NaN,NaN,NaN,https://a2.muscache.com/ac/pictures/105507202/...,https://a2.muscache.com/im/pictures/105507202/...,https://a2.muscache.com/ac/pictures/105507202/...,https://a2.muscache.com/ac/pictures/105507202/...,15830506,https://www.airbnb.com/users/show/15830506,Lize & Greg,2014-05-21,"Washington, District of Columbia, United States","We are two fun, friendly entrepreneurs living ...",within a few hours,92%,91%,f,https://a1.muscache.com/ac/users/15830506/prof...,https://a1.muscache.com/ac/users/15830506/prof...,Truxton Circle,26,26,"['email', 'phone', 'facebook', 'google', 'revi...",t,t,"3rd Street Southeast, Washington, DC 20003, Un...",Capitol Hill,"Capitol Hill, Lincoln Park",NaN,Washington,DC,20003,D.C.,"Washington, DC",US,United States,38.890046,-77.002808,t,House,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{TV,""Wireless Internet"",""Air Conditioning"",Kit...",NaN,$160.00,"$1,125.00","$5,225.00",$100.00,$115.00,1,$0.00,1,1125,today,t,0,0,8,283,2015-10-02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,18,NaN
1,975833,https://www.airbnb.com/rooms/975833,20151002231825,2015-10-03,Spacious Capitol Hill Townhouse,NaN,Beautifully renovated Capitol Hill townhouse. ...,Beautifully renovated Capitol Hill townhouse. ...,none,NaN,NaN,NaN,https://a2.muscache.com/ac/pictures/14613245/d...,https://a2.muscache.com/im/pictures/14613245/d...,https://a2.muscache.com/ac/pictures/14613245/d...,https://a2.muscache.com/ac/pictures/14613245/d...,5338703,https://www.airbnb.com/users/show/5338703,Sebastian,2013-03-05,"New York, New York, United States","Hi I'm Sebastian. \r\nAfter ten years in DC, m...",within a day,90%,100%,f,https://a1.muscache.com/ac/users/5338703/profi...,https://a1.muscache.com/ac/users/5338703/profi...,Capitol Hill,1,1,"['email', 'phone', 'reviews']",t,f,"11th Street Southeast, Washington, DC 20003, U...",Capitol Hill,"Capitol Hill, Lincoln Park",NaN,Washington,DC,20003,D.C.,"Washington, DC",US,United States,38.880413,-76.990485,t,House,Entire home/apt,6,3.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$350.00,NaN,NaN,NaN,$100.00,1,$0.00,2,30,today,t,12,38,68,343,2015-10-02,65,2013-03-22,2015-09-28,94.0,10.0,9.0,10.0,10.0,9.0,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,strict,f,f,1,2.11


In [4]:
# Selecting the columns we need
dc_listings = dc_listings[['id', 'host_id', 'host_response_rate', 'host_acceptance_rate',
       'host_listings_count', 'city', 'state', 'zipcode', 'market', 'latitude',
       'longitude', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'price', 'security_deposit', 'cleaning_fee', 'minimum_nights',
       'maximum_nights', 'number_of_reviews']]

In [5]:
print(dc_listings.columns)

Index(['id', 'host_id', 'host_response_rate', 'host_acceptance_rate',
       'host_listings_count', 'city', 'state', 'zipcode', 'market', 'latitude',
       'longitude', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'price', 'security_deposit', 'cleaning_fee', 'minimum_nights',
       'maximum_nights', 'number_of_reviews'],
      dtype='object')


#### Let's say we have a new listing which accommodates 3 people. 
We can compare our new listing with the market by calculating the Euclidean distance between each living space in the dataset and ours which accommodates 3 people.

In [6]:
# Calculate the distance between each value in the accommodates column from dc_listings and the 
# value 3, which is the number of people our listing accommodates. Calculate the unique value count.
new_listing = 3
dc_listings['distance'] = dc_listings['accommodates'].apply(lambda x: np.abs(x - new_listing))
print(dc_listings['distance'].value_counts())

1     2294
2      503
0      461
3      279
5       73
4       35
7       22
6       17
9       12
13       8
8        7
12       6
11       4
10       2
Name: distance, dtype: int64


#### Select "nearest neighbors" 
From the result, we can see there are 461 listings in the market that can accommodate 3 people just like ours. 

If now we want to select 5 "nearest neighbors", one thing we can do is, sort by the distance column and then just select the first 5 living spaces; However! we would be biasing the result to the original ordering of the dataset.

So, instead, randomize the ordering of the dataset and then sort the Dataframe by the distance column.


In [7]:
# see the number of rows in dc_listings
len(dc_listings)

3723

In [8]:
# provide a seed (i.e., the starting input) for NumPy’s pseudo-random number generator.
np.random.seed(1)
# shuffle the rows
dc_listings = dc_listings.loc[np.random.permutation(len(dc_listings))]

# sort by 'distance'
dc_listings = dc_listings.sort_values('distance')

# display the first 5 values in the price column
print(dc_listings.iloc[0:5]['price'])

577     $185.00
2166    $180.00
3631    $175.00
71      $128.00
1011    $115.00
Name: price, dtype: object


#### Price Prediction

We aimed to select 5 "nearest neighbors".
By calculating the mean of the first 5 values, we get out price prediction.
##### price prediction for our new listing =  (185 + 180 + 175 + 128 + 115) / 5 = 156.6



In [9]:
# cast price values from string to float
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

# Calculate the mean of the 5 nearest neighbors
nearest_neighbors = dc_listings.iloc[0:5]['price']
predicted_price = nearest_neighbors.mean()
print(predicted_price)

156.6


### Write a fuction for price prediction

In [10]:
dc_listings = pd.read_csv('listings.csv')

In [11]:
# cast price values from string to float
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

In [12]:
def predict_price(number_of_bedrooms):
    # To avoid reference error, use the DataFrame.copy() method so the underlying dataframe is assigned to temp_df,
    # instead of just a reference to dc_listings.
    temp_df = dc_listings.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(x - number_of_bedrooms))
    
    # provide a seed (i.e., the starting input) for NumPy’s pseudo-random number generator.
    np.random.seed(1)
    # shuffle the rows
    temp_df = temp_df.loc[np.random.permutation(len(dc_listings))]
    # sort by 'distance'
    temp_df = temp_df.sort_values('distance')
    
    # select the first 5 values in the price column
    nearest_neighbors = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbors.mean()
    return(predicted_price)

In [13]:
# Predict the price for our new listings on Airbnb

# a new listing that accommodates one person:
acc_one = predict_price(1)

acc_two = predict_price(2)
acc_three = predict_price(3)

print("Predicted price for acc_one: ${0:.2f}".format(acc_one))
print(acc_two)
print(acc_three)

Predicted price for acc_one: $68.00
112.8
156.6
